# Fourier Transform
### Discrete Fourier Transform, Fast Fourier Transform (DIT, DIF)

In [3]:
import numpy as np
import math
import matplotlib.pyplot as plt

pi=np.pi
e=np.e
inf=np.inf

## DFT
Discrete Fourier Transform


In [4]:
def DFT_forward(x):
    N=len(x)
    w = np.power(e, -2*pi*1j/N)
    F = []
    for i in range(N):
        F_=[]
        for j in range(N): F_.append(w**(i*j))
        F.append(F_)
    F = np.array(F)
    X = np.matmul(F,x)
    return X

def DFT_inverse(X):
    N = len(X)
    wc = np.power(e, 2*pi*1j/N)
    Fc = []
    for i in range(N):
        Fc_=[]
        for j in range(N): Fc_.append(wc**(i*j))
        Fc.append(Fc_)
    Fc = np.array(Fc)
    x = np.matmul(Fc,X)
    return (1/N)*x

## FFT : DIT
Fast Fourier Transform - Decimation in Time

In [5]:
def FFT_forw_time(x): # recursion :), pass np arrays
    N = len(x)
    if N%2>0 : raise ValueError(" N == 2**p : False")
    if N<=16: return DFT_forward(x)
    xe,xo=FFT_forw_time(x[::2]),FFT_forw_time(x[1::2]) # even and odd
    ws = np.exp(-2*pi*1j*np.arange(N)/N)
    return np.concatenate([xe + ws[: N//2] * xo , xe + ws[N//2 :] * xo])
    
def FFT_inv_time_util(X):
    N = len(X)
    if N%2>0 : raise ValueError(" N == 2**p : False")
    if N<=16: return N*DFT_inverse(X)
    Xe,Xo=FFT_inv_time_util(X[::2]),FFT_inv_time_util(X[1::2]) # even and odd
    ws = np.exp(2*pi*1j*np.arange(N)/N)
    return np.concatenate([Xe + ws[: N//2] * Xo , Xe + ws[N//2 :] * Xo])

def FFT_inv_time(X):
    N=len(X)
    return (1/N)*FFT_inv_time_util(X)

## FFT : DIF
Fast Fourier Transform - Decimation in Frequency

In [6]:
def FFT_forw_freq(x): # strictly pass numpy array
    N=len(x)
    if N%2 >0: raise ValueError(" N == 2**p : False")
    if N<=16: return DFT_forward(x)
    xl,xr = x[: N//2],x[N//2 :]
    Xe = FFT_forw_freq(xl+xr)
    wns = np.exp(-2*pi*1j*np.arange(N//2)/N)
    Xo = FFT_forw_freq((xl-xr)*wns)
    X = np.vstack((Xe,Xo)).ravel(order='F')
    return X
    
def FFT_inv_freq_util(X):
    N=len(X)
    if N%2 >0: raise ValueError(" N == 2**p : False")
    if N<=16: return N*DFT_inverse(X)
    Xl,Xr=X[:N//2],X[N//2:]
    xe = FFT_inv_freq_util(Xl+Xr)
    wns = np.exp(2*pi*1j*np.arange(N//2)/N)
    xo = FFT_inv_freq_util((Xl-Xr)*wns)
    x = np.vstack((xe,xo)).ravel(order='F')
    return x

def FFT_inv_freq(X):
    N=len(X)
    return (1/N)*FFT_inv_freq_util(X)



### Verify the code:

In [7]:
x=np.random.random(256)
X1=FFT_forw_freq(x)
X2=FFT_forw_time(x)
print(np.allclose(X1,np.fft.fft(x)))
print(np.allclose(X2,np.fft.fft(x)))

x1=FFT_inv_freq(X2)
x2=FFT_inv_time(X1)
print(np.allclose(x1,x))
print(np.allclose(x2,x))

True
True
True
True
